# **Dolly2.0 BB + Huggingfacing Embedding + Llamaindex**

V1. Tested with Colab free account. Resouce usage. 4.2G RAM. 6.5G CPU. 

In [2]:
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install datasets sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16

In [3]:
!pip -q install langchain transformers sentence_transformers llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.8/540.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 221.9 kB/s eta 0:00:00


In [4]:
from transformers import GenerationConfig, pipeline
import torch

In [5]:
from llama_index import SimpleDirectoryReader, GPTSimpleVectorIndex, PromptHelper, LLMPredictor, ServiceContext,  LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM

In [6]:
pipeline = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto" )

In [7]:
class customLLM(LLM):
    def _call(self, prompt, stop=None):
        res = pipeline(prompt)
        prompt_length = len(prompt)
#        print ("prompt_length =", prompt_length)
#        return res[0]["generated_text"][prompt_length:]         
        return res   

    def _identifying_params(self):
        return {"name_of_model": "dolly-v2-3b"}
    def _llm_type(self):
        return "custom"

Simple test the function of LLM

In [8]:
print(customLLM()._call("Tell me about New York City"))

New York is one of the most famous cities in the world. It is well known for it's diverse culture, diverse food, impressive architecture, and huge population of diverse ethnicity and religion. New York is also home to the famous Wall Street, Times square, and the so called Statue of Liberty. Many of the most famous companies, like Google, Walmart, Amazon and Dell have their headquarters in New York.


Setup the Llama Index solution

In [9]:
max_input_size = 512
num_output = 200
max_chunk_overlap = 20
chunk_size_limit = 200
llm_predictor = LLMPredictor(llm=customLLM())

In [10]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

In [11]:
prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap,chunk_size_limit=chunk_size_limit)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper, chunk_size_limit = chunk_size_limit) 

Create a folder under CONTENT called "data". Get a txt/csv file loaded for indexing. 

In [12]:
documents = SimpleDirectoryReader('./data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)
index.save_to_disk('index.json')

In [17]:
query_text = "How to setup milestones in project ?"
response = index.query(query_text,response_mode="compact",service_context=service_context, similarity_top_k=1)
print(response)

There are 4 ways to set a milestone in a project:
1. On-time
2. On-scope
3. By deadline
4. Any iteration
Depending on what you need to accomplish, you'll pick the best approach for you. 

If you just need to complete tasks with a set deadline, go with on-time or by-deadline approach. It's up to you to pick the best suitable approach based on your requirements.

If you need to achieve something beyond just a set of tasks with a specific approach at a specific point in time, you'll need a Scoping Exploration. A Scoping Exploration defines an explorative phase during project initiation, which will help you define the future state for the project. Explorative phase will help you think through various paths to explore for reducing the uncertainty in the project.
